# preprocess training data

In [1]:
import pandas as pd

# get the training dataset
train_data = pd.read_csv("./train.csv")

In [2]:
train_data.dropna(inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [3]:
# deal with purchase date
train_data["purchase_date"] = pd.to_datetime(train_data["purchase_date"])

# deal with release_date
train_data["release_date"] = pd.to_datetime(train_data["release_date"]) 

# get the different days between purchase_date and release_date
train_data["diff_day"] = train_data.apply(lambda raw: (raw["purchase_date"] - raw["release_date"]).days, axis=1)

# get the different years between purchase_date and release_date
train_data["diff_year"] = train_data["purchase_date"].dt.year - train_data["release_date"].dt.year

# delete purchase_date and release_date
train_data.drop(columns=["purchase_date", "release_date"], inplace=True)

train_data.head()

,id,playtime_forever,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,diff_day,diff_year
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",372.0,96.0,1665,5
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",23.0,0.0,472,1
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",3018.0,663.0,1616,4
3,3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",63078.0,1746.0,2434,6
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",8841.0,523.0,2043,6


In [4]:
# add negative rate and positive rate
train_data["negative_rate"] = train_data.apply(lambda raw: raw["total_negative_reviews"]\
                                               /(raw["total_negative_reviews"] + raw["total_positive_reviews"])\
                                               if raw["total_negative_reviews"] + raw["total_positive_reviews"] != 0\
                                               else 0.5, axis=1)
train_data["positive_rate"] = train_data.apply(lambda raw: raw["total_positive_reviews"]\
                                               /(raw["total_negative_reviews"] + raw["total_positive_reviews"])\
                                               if raw["total_negative_reviews"] + raw["total_positive_reviews"] != 0\
                                               else 0.5, axis=1)
train_data["total_reviews"] = train_data.apply(lambda raw: (raw["total_negative_reviews"] + raw["total_positive_reviews"]),\
                                               axis=1)

# delete total_positive_reviews and total_negative_reviews
train_data.drop(columns=["total_positive_reviews", "total_negative_reviews"], inplace=True)
train_data.head()

,id,playtime_forever,is_free,price,genres,categories,tags,diff_day,diff_year,negative_rate,positive_rate,total_reviews
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",1665,5,0.205128,0.794872,468.0
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",472,1,0.000000,1.000000,23.0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",1616,4,0.180114,0.819886,3681.0
3,3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2434,6,0.026934,0.973066,64824.0
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2043,6,0.055852,0.944148,9364.0


In [5]:
# deal with genres, categories, tags
genres = train_data["genres"].str.get_dummies(",")
categories = train_data["categories"].str.get_dummies(",") 
tags = train_data["tags"].str.get_dummies(",") 
finish_train_data = pd.concat([train_data, genres, categories, tags], axis=1)
finish_train_data.head()

,id,playtime_forever,is_free,price,genres,categories,tags,diff_day,diff_year,negative_rate,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",1665,5,0.205128,...,0,1,0,0,0,0,0,0,0,0
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",472,1,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",1616,4,0.180114,...,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2434,6,0.026934,...,0,0,0,0,0,0,0,0,0,0
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2043,6,0.055852,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# delete useless columns
finish_train_data.drop(columns=["id", "is_free", "genres", "categories", "tags"], inplace=True)
finish_train_data.head()

,playtime_forever,price,diff_day,diff_year,negative_rate,positive_rate,total_reviews,Action,Adventure,Animation & Modeling,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0.000000,3700.0,1665,5,0.205128,0.794872,468.0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0.016667,0.0,472,1,0.000000,1.000000,23.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,5000.0,1616,4,0.180114,0.819886,3681.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,9900.0,2434,6,0.026934,0.973066,64824.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.333333,4800.0,2043,6,0.055852,0.944148,9364.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# change price and total_reviews to type(int)
finish_train_data[["total_reviews", "price"]] = finish_train_data[["total_reviews", "price"]].astype("int")
finish_train_data.head()

,playtime_forever,price,diff_day,diff_year,negative_rate,positive_rate,total_reviews,Action,Adventure,Animation & Modeling,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0.000000,3700,1665,5,0.205128,0.794872,468,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0.016667,0,472,1,0.000000,1.000000,23,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,5000,1616,4,0.180114,0.819886,3681,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,9900,2434,6,0.026934,0.973066,64824,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.333333,4800,2043,6,0.055852,0.944148,9364,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# select several features
finish_train_data = finish_train_data[["playtime_forever", "price", "diff_day", \
             "negative_rate", "positive_rate", "total_reviews", "Difficult", "diff_year"]]
finish_train_data.head()

,playtime_forever,price,diff_day,negative_rate,positive_rate,total_reviews,Difficult,diff_year
0,0.000000,3700,1665,0.205128,0.794872,468,0,5
1,0.016667,0,472,0.000000,1.000000,23,0,1
2,0.000000,5000,1616,0.180114,0.819886,3681,0,4
3,1.533333,9900,2434,0.026934,0.973066,64824,0,6
4,22.333333,4800,2043,0.055852,0.944148,9364,0,6


# preprocess test data

In [9]:
import pandas as pd

# get the test dataset
test_data = pd.read_csv("./test.csv")

In [10]:
test_data["purchase_date"] = pd.to_datetime(test_data["purchase_date"])
test_data["release_date"] = pd.to_datetime(test_data["release_date"])

In [11]:
test_data["diff_day"] = test_data.apply(lambda raw: (raw["purchase_date"] - raw["release_date"]).days, axis=1)

# get the different years between purchase_date and release_date
test_data["diff_year"] = test_data["purchase_date"].dt.year - test_data["release_date"].dt.year

# delete purchase_date and release_date
test_data.drop(columns=["purchase_date", "release_date"], inplace=True)

test_data.head()

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,diff_day,diff_year
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2607.0,1122.0,2350.0,6.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",5762.0,2235.0,452.0,1.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",687.0,133.0,848.0,2.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",67.0,39.0,3850.0,11.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",40344.0,3708.0,1282.0,4.0


In [12]:
test_data["diff_year"].fillna(test_data["diff_year"].median(), inplace=True)
test_data["diff_day"].fillna(test_data["diff_day"].median(), inplace=True)
test_data["total_positive_reviews"].fillna(test_data["total_positive_reviews"].median(), inplace=True)
test_data["total_negative_reviews"].fillna(test_data["total_negative_reviews"].median(), inplace=True)
test_data.head()

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,diff_day,diff_year
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2607.0,1122.0,2350.0,6.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",5762.0,2235.0,452.0,1.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",687.0,133.0,848.0,2.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",67.0,39.0,3850.0,11.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",40344.0,3708.0,1282.0,4.0


In [13]:
# add negative rate and positive rate
test_data["negative_rate"] = test_data.apply(lambda raw: raw["total_negative_reviews"]\
                                               /(raw["total_negative_reviews"] + raw["total_positive_reviews"])\
                                               if raw["total_negative_reviews"] + raw["total_positive_reviews"] != 0\
                                               else 0.5, axis=1)
test_data["positive_rate"] = test_data.apply(lambda raw: raw["total_positive_reviews"]\
                                               /(raw["total_negative_reviews"] + raw["total_positive_reviews"])\
                                               if raw["total_negative_reviews"] + raw["total_positive_reviews"] != 0\
                                               else 0.5, axis=1)
test_data["total_reviews"] = test_data.apply(lambda raw: (raw["total_negative_reviews"] + raw["total_positive_reviews"]),\
                                               axis=1)

# delete total_positive_reviews and total_negative_reviews
test_data.drop(columns=["total_positive_reviews", "total_negative_reviews"], inplace=True)
test_data.head()

,id,is_free,price,genres,categories,tags,diff_day,diff_year,negative_rate,positive_rate,total_reviews
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2350.0,6.0,0.300885,0.699115,3729.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",452.0,1.0,0.279480,0.720520,7997.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",848.0,2.0,0.162195,0.837805,820.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",3850.0,11.0,0.367925,0.632075,106.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",1282.0,4.0,0.084173,0.915827,44052.0


In [14]:
# deal with genres, categories, tags
genres = test_data["genres"].str.get_dummies(",")
categories = test_data["categories"].str.get_dummies(",") 
tags = test_data["tags"].str.get_dummies(",") 
finish_test_data = pd.concat([test_data, genres, categories, tags], axis=1)
finish_test_data.head()

,id,is_free,price,genres,categories,tags,diff_day,diff_year,negative_rate,positive_rate,...,Violent,Visual Novel,Walking Simulator,War,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2350.0,6.0,0.300885,0.699115,...,0,0,0,0,0,0,0,0,0,0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",452.0,1.0,0.279480,0.720520,...,0,0,0,0,0,0,0,0,0,0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",848.0,2.0,0.162195,0.837805,...,0,0,0,0,0,0,0,0,0,0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",3850.0,11.0,0.367925,0.632075,...,0,0,0,0,0,0,0,0,0,0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",1282.0,4.0,0.084173,0.915827,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# delete useless columns
finish_test_data.drop(columns=["id", "is_free", "genres", "categories", "tags"], inplace=True)
finish_test_data.head()

,price,diff_day,diff_year,negative_rate,positive_rate,total_reviews,Action,Adventure,Casual,Early Access,...,Violent,Visual Novel,Walking Simulator,War,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,3500,2350.0,6.0,0.300885,0.699115,3729.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11600,452.0,1.0,0.279480,0.720520,7997.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2100,848.0,2.0,0.162195,0.837805,820.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3600,3850.0,11.0,0.367925,0.632075,106.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3400,1282.0,4.0,0.084173,0.915827,44052.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# change price and total_reviews to type(int)
finish_test_data[["total_reviews", "diff_day", "diff_year"]] = finish_test_data[["total_reviews", "diff_day", "diff_year"]].astype("int")
finish_test_data.head()

,price,diff_day,diff_year,negative_rate,positive_rate,total_reviews,Action,Adventure,Casual,Early Access,...,Violent,Visual Novel,Walking Simulator,War,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,3500,2350,6,0.300885,0.699115,3729,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11600,452,1,0.279480,0.720520,7997,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2100,848,2,0.162195,0.837805,820,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3600,3850,11,0.367925,0.632075,106,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3400,1282,4,0.084173,0.915827,44052,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# select several features
finish_test_data = finish_test_data[["price", "diff_day", \
             "negative_rate", "positive_rate", "total_reviews", "Difficult", "diff_year"]]
finish_test_data.head()

,price,diff_day,negative_rate,positive_rate,total_reviews,Difficult,diff_year
0,3500,2350,0.300885,0.699115,3729,0,6
1,11600,452,0.279480,0.720520,7997,1,1
2,2100,848,0.162195,0.837805,820,0,2
3,3600,3850,0.367925,0.632075,106,0,11
4,3400,1282,0.084173,0.915827,44052,0,4


In [18]:
finish_test_data

,price,diff_day,negative_rate,positive_rate,total_reviews,Difficult,diff_year
0,3500,2350,0.300885,0.699115,3729,0,6
1,11600,452,0.279480,0.720520,7997,1,1
2,2100,848,0.162195,0.837805,820,0,2
3,3600,3850,0.367925,0.632075,106,0,11
4,3400,1282,0.084173,0.915827,44052,0,4
...,...,...,...,...,...,...,...
85,5000,1766,0.052386,0.947614,9850,0,5
86,3600,3092,0.123856,0.876144,1421,0,9
87,11200,1650,0.160849,0.839151,39478,0,5
88,0,153,0.000000,1.000000,5,0,0


# Fit model

In [19]:
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import numpy as np

In [20]:
X_train = finish_train_data.drop(columns=["playtime_forever"])
Y_train = finish_train_data["playtime_forever"]
X_test = finish_test_data

In [21]:
##################### LightGBM model #################################
# param_test1 = {'n_estimators':list(range(10, 210, 10))}
# gsearch1 = GridSearchCV(estimator = lgb.LGBMRegressor(), 
#                    param_grid = param_test1, cv=5)
# model = gsearch1.fit(X_train, Y_train)

# best_model = model.best_estimator_

# test = best_model.predict(X_test)

model = lgb.LGBMRegressor(objective='regression', n_estimators=20)
# rf = RandomForestRegressor()
model.fit(X_train, Y_train)

test = model.predict(X_test)

In [22]:
from pandas.core.frame import DataFrame

submit = {"playtime_forever": test}
submit_data = DataFrame(submit)
submit_data["playtime_forever"].to_list()

[0.769778903339183,
 4.044807792703117,
 0.6016961877708269,
 0.41870725560536776,
 1.7422730146299803,
 0.04227312881992268,
 0.43533307342385397,
 0.5414789305857963,
 0.9989875266912475,
 1.4289861507092745,
 1.2081901952479315,
 0.9640745827936087,
 0.9708932349220025,
 3.15338643322236,
 5.100526241999446,
 2.5541852408506034,
 2.671777319483349,
 2.213959486901301,
 2.995622474214846,
 0.9873031938600882,
 0.5752681852850143,
 0.4488499969665478,
 0.8112697304684017,
 0.7798757065805128,
 1.5037165515085023,
 0.15144793294266246,
 0.3992765920862864,
 0.7744167548550266,
 0.820943639035226,
 0.3488275680901297,
 0.9447903169151445,
 12.887493357050575,
 2.7482641920597923,
 0.6515159419073981,
 1.9750708146302787,
 0.08801817838808113,
 1.5651346630365515,
 2.683009359347039,
 2.995622474214846,
 1.0969532818273513,
 1.884931990970124,
 1.923202483446439,
 2.3747237386228885,
 0.6398355323674598,
 3.21627694857225,
 0.6177258792381339,
 1.6543316047802343,
 1.659011135244315,
 1.

In [23]:
submit_data.to_csv("./final_1.csv", index_label="id")